In [76]:
import json
import pandas as pd
from konlpy.tag import  Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import numpy as np
import os

In [77]:
# # 월별 데이터로 생성하기
# json_list = os.listdir('../news-crawler/it/')
# json_files = [file for file in json_list if file.endswith('.json')]  
# data = []
# df = pd.DataFrame()
# for i in json_files:
#     for line in open(('../news-crawler/it/'+i),"r", encoding='utf-8-sig'):
#         df = pd.concat([df, pd.DataFrame(json.loads(line), columns=['id', 'title', 'content', 'date', 'like'])])  

In [78]:
# 주차별 데이터로 생성하기
with open("../news-crawler/politics/2021-11-16-20.json", "r", encoding='utf-8-sig') as f:
    tmp = json.load(f)
df = pd.DataFrame(tmp, columns=['id', 'title', 'content', 'date', 'like'])

In [88]:
okt = Okt() 
# okt.analyze  #구(Phrase) 분석
# okt.morphs   #형태소 분석
# okt.nouns    #명사 분석
# okt.pos      #형태소 분석 태깅
titles_list = []
for title in tqdm(df['title']): 
    check3 = "\'\",”“’‘"
    for check in check3:
        title = title.replace(check, '')
    titles_list.append(title)
    print(title)
df['titles'] = titles_list
    
noun_list = []
for content in tqdm(df['content']): 
    nouns = okt.nouns(content)
    noun_list.append(nouns)
df['nouns'] = noun_list
print(df.head())

100%|██████████| 301/301 [00:00<00:00, 27327.71it/s]

이제 지쳤다 황교익 돌연 이재명에 선 긋는 이유
황교익 이재명 지지한 적 없어…文 지지선언 후 온갖 구설 시달려
GTX-A 노선·동탄급행철도 연결하는 삼성역 개통 7년 지연 이유는?
GTX-A 운정~서울역 삼성~동탄 구간 연결 10개월 앞당긴다
마지막 공익근무라 생각…사람이 먼저다 文슬로건 만든 정철 이재명 캠프 합류
동탄2신도시 입주민 2027년까지 GTX-A로 삼성역 못 간다
산책 의혹 후폭풍…사실 왜곡 스스로 논란 키워
계기되면 도울 수도 김종인의 여지…尹 선대위 언제쯤?
기민함 부족 이재명 연일 당정 때리기…전형적 남 탓
GTX-A 삼성역 2028년에나 문 연다…파주~서울역 수서~동탄 분리
황교익 이재명 지지한적 없는데…문재인 지지자까지 날 팼다
사람이 먼저다 슬로건 만든 정철 文 이어 이재명 돕는다
보건전문기구 대표 만난 文 글로벌 백신 인력 양성 허브 유치(종합)
文 우리 기술로 백신 생산 국가적 목표…SK바이오 기대 커
감사원 GTX-A 핵심 삼성역 2028년에나 개통
문대통령 韓 백신개발 관심을 CEPI 선도국가 조건 갖춰(종합)
文대통령 SK바사 백신 개발 관심을… 해쳇 韓 과학기술 허브 정평
감사원 GTX-A 2024년 개통 가능한데 삼성역은 2028년에나…
황교익 이재명 지지한 적 없다…시민의 정치적 발언일 뿐
황교익 이재명 지지한 적 없다…文지지 선언 후 온갖 구설 시달려
文 글로벌 백신허브 뛰어넘어 우리 기술로 백신 개발·생산...국가 목표
감사원 국토부 GTX 삼성역 개통 지연 방치…특단조치 마련하라
감사원 서울시 공사 지연…GTX-A 28년까지 삼성역 못 서
文 사람이 먼저다 정철 이재명 슬로건 맡아…李와 尹은 품질에서 큰 차이
文대통령 SK바이오사이언스 코로나 백신 2상 대단히 성공적
文대통령 우리 기술로 백신 개발·생산하는 게 국가적 목표
황교익 돌연 지지 철회?…이재명 지지한 적 없어
문 대통령 고유 기술로 한국 백신 생산·개발이 목표
황교익 이재명 지지한 적 없어…온갖 구설에 시달리는 중
文 우리 백신 개발 국가목표…감염병 혁

In [80]:
# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

vector = np.array(vector) # Normalizer를 이용해 변환된 벡터
model = DBSCAN(eps=0.3,min_samples=6, metric = "cosine")
# 거리 계산 식으로는 Cosine distance를 이용
result = model.fit_predict(vector)
df['result'] = result

In [81]:
# Delete noise or garbage data
df = df[df['result'] != -1]
df = df[df['result'] != 0]

cluster_dict = {}

# Create clustering using titles
for i in df['result'].unique().tolist():
    cluster_dict[i] = df[df['result'] == i].titles.tolist()

# Order by Hot topic    
sorted_cluster_dict = sorted(cluster_dict.items(), key=lambda x : len(x[1]), reverse=True)

In [82]:
final_dict = {}

# Create 10 data sets of clustered data
for idx, (cluster_num, titles) in enumerate(sorted_cluster_dict[:10]):
    keywords_list = []
    id_list = []
    like_list = []

    # Extract keyword from title
    for title in titles:
        res = " ".join(titles).split()
        keywords = okt.nouns(title)
        keywords_list.extend(res)
        # Add like / ID for sorting and linking
        id_list = df[df['result']==cluster_num].id.tolist()
        like_list = df[df['result']==cluster_num].like.tolist()
    # Create keyword standard
    standard_keyword = set(keywords_list)
    print(standard_keyword)
    tmp_keyword = []

    # Keyword Counting
    for keyword in standard_keyword:
        tmp_keyword.append((keyword, keywords_list.count(keyword)))

    # Hot Keyword Top 3
    tmp_keyword.sort(key = lambda x : x[1], reverse=True)
    final_keyword = [keyword for keyword in tmp_keyword[:5]]
    print(final_keyword)
    # Create dictonary file
    final_data = []
    for id, title, like in zip(id_list, titles, like_list):
        id['like'] = like
        id['title'] = title
        final_data.append(id)
    final_dict[f"cluster{idx}"] = {}
    final_dict[f'cluster{idx}']['keyword'] = final_keyword
    final_dict[f'cluster{idx}']['data'] = final_data

{'의혹', '이재명', '관련', '李,', '조건부', '논란', '도울', '분노', '김혜경', '직격', '지령', '우리가', '…야당', '남', '돼야', '에', '부족', '…선대위', '토론회…어떤', '뿌리는', '비판', '화천대유', '키워', '수사', '연일', '해야', '이야기', '위험', '시간', '잇겠다', '못한', '채', '지지율', '추격자', '초보운전이', '신경전도', '윤석열,', '미진하면…尹도', '좌표', '풀릴까?', '벌렸다', '대장동', '음주운전보다', '정치', '최장', '가짜뉴스', '수도', '때리기…', '괴소문', '공개…이-윤', '지속…이재명과', '강력', '언제쯤?', '특검', '국민', '김종인에', '스스로', '유포', '상승세', '…송영길', '적개심의', '구애', 'CCTV', '격차', '분향', '대응', '김종인의', '산책', '수용', '일정', '부인', '조치', '백브리핑', '후폭풍…', '기민함', '사실', '법적', '통합', '선대위', '당정', '…호남', '사과', '윤석열', '직격…', '與', '계기되면', '관훈클럽', '더', '지지자에', '재개…', '언론', '왜곡', '낙상사고', '5·18', '탓', '이재명,', '오갔나', '허위사실', '공개', '된', '전형적', '언제까지?', '여지…尹', '尹', '밀당'}
[('이재명', 75), ('윤석열', 60), ('관련', 30), ('윤석열,', 30), ('부인', 30)]
{'[전문]文,', 'FLNG', '만나', '모두', '거제', '크기', '우리가', '술(Coral', '대통령', '대통령,', '여정에', '출항…文', '출항·명명식', 'LNG', '4개', '세계최고', '많은', '주목…재생에너지', 'FLNG선', '출항', 'Sul', '호...“탄소중립', '韓·모잠비크', '[전문]', '명명식', '출항에', '文대통령', 

In [83]:
final_dict

{'cluster0': {'keyword': [('이재명', 75),
   ('윤석열', 60),
   ('관련', 30),
   ('윤석열,', 30),
   ('부인', 30)],
  'data': [{'company': '055',
    'id': '0000933782',
    'sid': '100',
    'like': 17,
    'title': ' 산책 의혹  후폭풍… 사실 왜곡   스스로 논란 키워 '},
   {'company': '055',
    'id': '0000933777',
    'sid': '100',
    'like': 1,
    'title': ' 계기되면 도울 수도  김종인의 여지…尹 선대위 언제쯤?'},
   {'company': '055',
    'id': '0000933775',
    'sid': '100',
    'like': 288,
    'title': ' 기민함 부족  이재명 연일 당정 때리기… 전형적 남 탓 '},
   {'company': '055',
    'id': '0000933581',
    'sid': '100',
    'like': 29,
    'title': '윤석열 지지율 상승세 지속…이재명과 격차 더 벌렸다'},
   {'company': '055',
    'id': '0000933568',
    'sid': '100',
    'like': 1367,
    'title': '이재명, 지지자에  우리가 언론 돼야 …야당  좌표 지령 '},
   {'company': '055',
    'id': '0000933567',
    'sid': '100',
    'like': 2,
    'title': '윤석열, 김종인에  공개 구애 …선대위  밀당  언제까지?'},
   {'company': '055',
    'id': '0000933566',
    'sid': '100',
    'like': 67,
    'title': ' 추격자  된 이재명, 尹 직격… 화

In [84]:
file_path = '../json_data/it/new_data.json'
with open(file_path, 'w', encoding='utf-8') as f:
	json.dump(final_dict, f, ensure_ascii=False)